In [1]:
import json

In [3]:
import graphlab as gl

In [6]:
import requests

In [7]:
from requests_file import FileAdapter

s = requests.Session()
s.mount('file://', FileAdapter())

resp = s.get('file:///users/ty/code/getmoore/data/us-west-1-spot.json')

In [9]:
pricing_history = resp.json()

In [20]:
pricing_history = pricing_history['SpotPriceHistory']

In [24]:
pricing_history[0]

{u'AvailabilityZone': u'us-west-1a',
 u'InstanceType': u'c4.large',
 u'ProductDescription': u'SUSE Linux',
 u'SpotPrice': u'0.120100',
 u'Timestamp': u'2016-09-21T00:50:37.000Z'}

In [28]:
sf = gl.SFrame(data=pricing_history)

In [30]:
sf.head()

X1
"{'Timestamp': '2016-09-21T00:50:37.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:37.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:32.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:32.000Z', ..."


In [35]:
counts = {'Timestamp': 0, 'InstanceType': 0, 'ProductDescription': 0, 'SpotPrice': 0, 'Timestamp': 0}
for spot in pricing_history:
    keys = spot.keys()
    for key in keys:
        if key not in counts.keys():
            counts[key] = 0
        else:
            counts[key] += 1

In [36]:
counts

{u'AvailabilityZone': 2369078,
 'InstanceType': 2369079,
 'ProductDescription': 2369079,
 'SpotPrice': 2369079,
 'Timestamp': 2369079}

In [38]:
# All but one of the rows has the 'AvailabilityZone' key

In [41]:
timestamp, instance_type, product_description, spot_price, availability_zone = [], [], [], [], []

In [42]:
for spot in pricing_history:
    keys = spot.keys()
    if 'AvailabilityZone' not in keys:
        availability_zone.append(None)
    else:
        timestamp.append(spot['Timestamp'])
        instance_type.append(spot['InstanceType'])
        product_description.append(spot['ProductDescription'])
        spot_price.append(spot['SpotPrice'])
        availability_zone.append(spot['AvailabilityZone'])

In [43]:
len(timestamp)

2369079

In [44]:
len(instance_type)

2369079

In [45]:
len(product_description)

2369079

In [46]:
len(spot_price)

2369079

In [47]:
len(availability_zone)

2369079

In [48]:
# Load the SFrame. Note, changing SpotPrice to Price as this dataset only contains spot prices
sf = gl.SFrame({'Timestamp': timestamp, 'InstanceType': instance_type, 
                'ProductDescription': product_description, 'Price': spot_price, 
                'AvailabilityZone': availability_zone})

In [49]:
sf.head()

AvailabilityZone,InstanceType,Price,ProductDescription,Timestamp
us-west-1a,c4.large,0.120100,SUSE Linux,2016-09-21T00:50:37.000Z
us-west-1a,c4.large,0.020100,Linux/UNIX,2016-09-21T00:50:37.000Z
us-west-1b,m2.4xlarge,0.213300,SUSE Linux,2016-09-21T00:50:36.000Z
us-west-1b,m2.4xlarge,0.113300,Linux/UNIX,2016-09-21T00:50:36.000Z
us-west-1b,c4.large,0.100300,Windows,2016-09-21T00:50:36.000Z
us-west-1b,c3.2xlarge,0.265600,Windows,2016-09-21T00:50:35.000Z
us-west-1b,c3.2xlarge,0.254200,SUSE Linux,2016-09-21T00:50:35.000Z
us-west-1b,c3.2xlarge,0.154200,Linux/UNIX,2016-09-21T00:50:35.000Z
us-west-1b,r3.2xlarge,0.850000,SUSE Linux,2016-09-21T00:50:32.000Z
us-west-1b,r3.2xlarge,0.750000,Linux/UNIX,2016-09-21T00:50:32.000Z


In [50]:
gl.canvas.set_target('browser')

In [52]:
sf.show()

Canvas is accessible via web browser at the URL: http://localhost:52382/index.html
Opening Canvas in default web browser.


In [53]:
# See if c4.8xlarge was ever available for under $0.10 / hr
c48xlarge = sf[sf['InstanceType'] == 'c4.8xlarge']

In [54]:
c48xlarge.head()

AvailabilityZone,InstanceType,Price,ProductDescription,Timestamp
us-west-1b,c4.8xlarge,0.487000,SUSE Linux,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,0.387000,Linux/UNIX,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,1.587300,Windows,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,1.587200,Windows,2016-09-21T00:48:20.000Z
us-west-1b,c4.8xlarge,0.485900,SUSE Linux,2016-09-21T00:48:19.000Z
us-west-1b,c4.8xlarge,0.385900,Linux/UNIX,2016-09-21T00:48:19.000Z
us-west-1a,c4.8xlarge,1.648100,Windows,2016-09-21T00:47:25.000Z
us-west-1b,c4.8xlarge,1.587300,Windows,2016-09-21T00:47:25.000Z
us-west-1a,c4.8xlarge,0.538400,SUSE Linux,2016-09-21T00:47:25.000Z
us-west-1b,c4.8xlarge,0.487000,SUSE Linux,2016-09-21T00:47:25.000Z


In [55]:
min(c48xlarge['Price'])

'0.256000'

Yikes, need to up that hashrate!

In [56]:
c48xlarge.show()

Canvas is accessible via web browser at the URL: http://localhost:52382/index.html
Opening Canvas in default web browser.


In [58]:
sf.save('data/us-west-1-spot-clean.json', format='json')
sf.save('data/us-west-1-spot-clean.csv', format='csv')